In [1]:
import pandas as pd

def calculate_atm_strike_price_for_date(futures_df, selected_date):
    """
    This function calculates the ATM strike price for a given date at 10:29:59 am.

    Parameters:
    futures_df (pd.DataFrame): Input DataFrame with 'Date', 'Time', and 'Close' columns
    selected_date (str): The date for which to calculate the ATM strike price (in 'YYYY-MM-DD' format)

    Returns:
    float or None: The ATM strike price or None if the date/time is not found
    """
    
    # Ensure 'Time' column is in datetime format
    futures_df['Time'] = pd.to_datetime(futures_df['Time'], format='%H:%M:%S').dt.time
    futures_df['Date'] = pd.to_datetime(futures_df['Date'], format='%Y-%m-%d')

    # Filter data for the specified date and 10:29:59 am
    trade_time = pd.to_datetime('10:29:59', format='%H:%M:%S').time()
    filtered_df = futures_df[(futures_df['Date'] == pd.to_datetime(selected_date, format='%Y-%m-%d')) &
                             (futures_df['Time'] == trade_time)]
    
    # Check if we have at least one row after filtering
    if not filtered_df.empty:
        # Calculate the average close price and determine the ATM strike price
        avg_close_price = filtered_df['Close'].mean()
        atm_strike_price = round(avg_close_price / 100) * 100
        return atm_strike_price
    else:
        # Return None if no rows are found for the specified date and time
        return None

In [2]:
def log_trades(trade_log, entry_date, expiry_date, entry_time, atm_strike_price, call_options_atm, put_options_atm, call_options_otm, put_options_otm, capital, stop_loss, target):
    """
    This function logs trades by appending the options data to the trade log.

    Parameters:
    trade_log (list): List to store the trade log entries
    entry_date (str): The date at which the trade was entered
    expiry_date (str): The expiry date of the options
    entry_time (str): The time at which the trade was entered
    atm_strike_price (float): The ATM strike price for the selected date
    call_options_atm (pd.Series): Series containing the ATM call options data
    put_options_atm (pd.Series): Series containing the ATM put options data
    call_options_otm (pd.Series): Series containing the OTM call options data
    put_options_otm (pd.Series): Series containing the OTM put options data
    capital (float): The total capital available for trading
    stop_loss (float): Calculated stop loss
    target (float): Calculated target

    Returns:
    list: Updated trade log
    """
    trade_log.append({
        'Serial_No': len(trade_log) + 1,
        'Date': entry_date,
        'Expiry': expiry_date,
        'En_Date': entry_date,
        'Ce_En_Date': entry_date,
        'Pe_En_Date': entry_date,
        'Ce_Ex_Date': None,  # Placeholder for call exit date
        'Pe_Ex_Date': None,  # Placeholder for put exit date
        'Atm_Strike': atm_strike_price,
        'Ce_Short_Strike': call_options_atm['Strike'],
        'Pe_Short_Strike': put_options_atm['Strike'],
        'Ce_Long_Strike': call_options_otm['Strike'],
        'Pe_Long_Strike': put_options_otm['Strike'],
        'Ce_Short_En_Price': call_options_atm['Close'],
        'Ce_Short_Ex_Price': None,  # Placeholder for call short exit price
        'Pe_Short_En_Price': put_options_atm['Close'],
        'Pe_Short_Ex_Price': None,  # Placeholder for put short exit price
        'Ce_Long_En_Price': call_options_otm['Close'],
        'Ce_Long_Ex_Price': None,  # Placeholder for call long exit price
        'Pe_Long_En_Price': put_options_otm['Close'],
        'Pe_Long_Ex_Price': None,  # Placeholder for put long exit price
        'Entry_Time': entry_time,
        'Ticker_Ce_Short': call_options_atm['Ticker'],
        'Ticker_Pe_Short': put_options_atm['Ticker'],
        'Ticker_Ce_Long': call_options_otm['Ticker'],
        'Ticker_Pe_Long': put_options_otm['Ticker'],
        'Exit_Reason': None,  # Placeholder for exit reason
        'Stop_Loss': stop_loss,
        'Total_Premium_Received': call_options_atm['Close'] + put_options_atm['Close'] - call_options_otm['Close'] - put_options_otm['Close'],
        'Target': target,
        'Total_PnL': None  # Placeholder for total P&L
    })
    return trade_log


In [3]:
def create_straddle(options_df, futures_df, selected_date, entry_time, capital):
    """
    This function creates a straddle by taking an entry in options for a specific date using the ATM strike price.
    It also adds 2% away wings for protection by buying both an out-of-the-money call option and an out-of-the-money put option.
    It logs the trades in a trade log.

    Parameters:
    options_df (pd.DataFrame): DataFrame containing options data
    futures_df (pd.DataFrame): DataFrame containing futures data
    selected_date (str): The date for which to create the straddle (in 'YYYY-MM-DD' format)
    entry_time (str): The time at which the trade is entered (in 'HH:MM:SS' format)
    capital (float): The total capital available for trading

    Returns:
    list: Updated trade log
    """

    # Calculate the ATM strike price for the selected date
    atm_strike_price = calculate_atm_strike_price_for_date(futures_df, selected_date)
    if atm_strike_price is None:
        print(f"No ATM strike price found for {selected_date} at {entry_time}.")
        return []

    # Ensure 'Date' and 'Time' columns are in datetime format
    options_df['Date'] = pd.to_datetime(options_df['Date'], format='%Y-%m-%d')
    options_df['Expiry'] = pd.to_datetime(options_df['Expiry'], format='%Y-%m-%d')
    options_df['Time'] = pd.to_datetime(options_df['Time'], format='%H:%M:%S').dt.time

    # Filter options data for the specified date and entry time
    entry_time_dt = pd.to_datetime(entry_time).time()
    options_for_date = options_df[(options_df['Date'] == pd.to_datetime(selected_date, format='%Y-%m-%d')) &
                                  (options_df['Time'] == entry_time_dt)]

    if options_for_date.empty:
        print(f"No options data found for {selected_date} at {entry_time}.")
        return []

    # Find the closest expiry date to the selected date
    closest_expiry = options_for_date['Expiry'].min()

    # Filter options data for the closest expiry date
    options_for_closest_expiry = options_for_date[options_for_date['Expiry'] == closest_expiry]

    # Calculate the 2% away strikes and round them to the nearest 100 points
    strike_increment = atm_strike_price * 0.02
    otm_call_strike = round((atm_strike_price + strike_increment) / 100) * 100
    otm_put_strike = round((atm_strike_price - strike_increment) / 100) * 100

    # Separate call and put options for the ATM and OTM strikes
    call_options_atm = options_for_closest_expiry[(options_for_closest_expiry['Type'] == 'CE') &
                                                  (options_for_closest_expiry['Strike'] == atm_strike_price)]
    put_options_atm = options_for_closest_expiry[(options_for_closest_expiry['Type'] == 'PE') &
                                                 (options_for_closest_expiry['Strike'] == atm_strike_price)]
    call_options_otm = options_for_closest_expiry[(options_for_closest_expiry['Type'] == 'CE') &
                                                  (options_for_closest_expiry['Strike'] == otm_call_strike)]
    put_options_otm = options_for_closest_expiry[(options_for_closest_expiry['Type'] == 'PE') &
                                                 (options_for_closest_expiry['Strike'] == otm_put_strike)]

    if call_options_atm.empty or put_options_atm.empty or call_options_otm.empty or put_options_otm.empty:
        print(f"Not all required options found for strikes {atm_strike_price}, {otm_call_strike}, {otm_put_strike} on {selected_date} at {entry_time}.")
        return []

    # Calculate stop loss and target
    total_entry_premium = call_options_atm.iloc[0]['Close'] + put_options_atm.iloc[0]['Close'] - call_options_otm.iloc[0]['Close'] - put_options_otm.iloc[0]['Close']
    stop_loss = total_entry_premium * 1.3
    target = total_entry_premium * 0.2

    # Initialize trade log
    trade_log = []

    # Log the trades
    trade_log = log_trades(trade_log, selected_date, closest_expiry, entry_time, atm_strike_price, call_options_atm.iloc[0], put_options_atm.iloc[0], call_options_otm.iloc[0], put_options_otm.iloc[0], capital, stop_loss, target)

    return trade_log


In [4]:
def filter_options_data(trade_log, options_df):
    """
    This function filters the options data for each trade in the trade log and combines them into a single DataFrame.
    """
    filtered_data = []
    for _, trade in trade_log.iterrows():
        ce_short_data = options_df[(options_df['Strike'] == trade['Ce_Short_Strike']) & 
                                   (options_df['Expiry'] == trade['Expiry']) & 
                                   (options_df['Type'] == 'CE') & 
                                   (options_df['Date'] >= pd.to_datetime(trade['Date']))]
        
        pe_short_data = options_df[(options_df['Strike'] == trade['Pe_Short_Strike']) & 
                                   (options_df['Expiry'] == trade['Expiry']) & 
                                   (options_df['Type'] == 'PE') & 
                                   (options_df['Date'] >= pd.to_datetime(trade['Date']))]
        
        ce_long_data = options_df[(options_df['Strike'] == trade['Ce_Long_Strike']) & 
                                  (options_df['Expiry'] == trade['Expiry']) & 
                                  (options_df['Type'] == 'CE') & 
                                  (options_df['Date'] >= pd.to_datetime(trade['Date']))]
        
        pe_long_data = options_df[(options_df['Strike'] == trade['Pe_Long_Strike']) & 
                                  (options_df['Expiry'] == trade['Expiry']) & 
                                  (options_df['Type'] == 'PE') & 
                                  (options_df['Date'] >= pd.to_datetime(trade['Date']))]
        
        combined_data = pd.concat([ce_short_data, pe_short_data, ce_long_data, pe_long_data]).sort_values(by=['Date', 'Time'])
        filtered_data.append(combined_data)
        
    return pd.concat(filtered_data).reset_index(drop=True)


In [5]:
def monitor_and_exit_trades(trade_log_df, options_df):
    """
    This function monitors the trades in the trade log and exits them based on the specified conditions:
    1. A 30% stop loss based on the premium collected from selling the straddle.
    2. An 80% target.
    3. If neither condition is met, hold the position until 15:19:59 on the expiry date and exit all legs of the trade together.

    Parameters:
    trade_log_df (pd.DataFrame): DataFrame containing the trade log entries
    options_df (pd.DataFrame): DataFrame containing options data

    Returns:
    pd.DataFrame: Updated trade log with exit prices and times
    """

    options_df['Date'] = pd.to_datetime(options_df['Date'], format='%Y-%m-%d')
    options_df['Expiry'] = pd.to_datetime(options_df['Expiry'], format='%Y-%m-%d')
    options_df['Time'] = pd.to_datetime(options_df['Time'], format='%H:%M:%S').dt.time
    options_df['DateTime'] = pd.to_datetime(options_df['Date'].astype(str) + ' ' + options_df['Time'].astype(str))

    filtered_options_df = filter_options_data(trade_log_df, options_df)

    for index, trade in trade_log_df.iterrows():
        entry_datetime = pd.to_datetime(f"{trade['Date']} {trade['Entry_Time']}")
        expiry_datetime = pd.to_datetime(f"{trade['Expiry']} 15:19:59")

        options_contract_data = filtered_options_df[(filtered_options_df['Expiry'] == trade['Expiry']) & 
                                                    (filtered_options_df['DateTime'] >= entry_datetime) & 
                                                    (filtered_options_df['DateTime'] <= expiry_datetime)]

        # Initialize stop loss, target, and premium received
        premium_collected = trade['Ce_Short_En_Price'] + trade['Pe_Short_En_Price'] - trade['Ce_Long_En_Price'] - trade['Pe_Long_En_Price']
        stop_loss = premium_collected * 1.30
        target = premium_collected * 0.20
        total_premium_received = premium_collected
        exit_reason = 'Held till expiry'
        total_pnl = None
        ce_short_exit_price = None
        pe_short_exit_price = None
        ce_long_exit_price = None
        pe_long_exit_price = None
        exit_time = None

        for current_datetime, group in options_contract_data.groupby('DateTime'):
            ce_short_close = group[(group['Strike'] == trade['Ce_Short_Strike']) & (group['Type'] == 'CE')]['Close']
            pe_short_close = group[(group['Strike'] == trade['Pe_Short_Strike']) & (group['Type'] == 'PE')]['Close']
            ce_long_close = group[(group['Strike'] == trade['Ce_Long_Strike']) & (group['Type'] == 'CE')]['Close']
            pe_long_close = group[(group['Strike'] == trade['Pe_Long_Strike']) & (group['Type'] == 'PE')]['Close']

            if not (ce_short_close.empty or pe_short_close.empty or ce_long_close.empty or pe_long_close.empty):
                ce_short_close = ce_short_close.values[0]
                pe_short_close = pe_short_close.values[0]
                ce_long_close = ce_long_close.values[0]
                pe_long_close = pe_long_close.values[0]

                total_premium = (trade['Ce_Short_En_Price'] + trade['Pe_Short_En_Price'] - trade['Ce_Long_En_Price'] - trade['Pe_Long_En_Price']) - (ce_short_close + pe_short_close - ce_long_close - pe_long_close)

                if total_premium <= -stop_loss:
                    exit_reason = 'Stop Loss Hit'
                    total_pnl = total_premium
                    ce_short_exit_price = ce_short_close
                    pe_short_exit_price = pe_short_close
                    ce_long_exit_price = ce_long_close
                    pe_long_exit_price = pe_long_close
                    exit_time = current_datetime
                    break

                if total_premium >= target:
                    exit_reason = 'Target Achieved'
                    total_pnl = total_premium_received - total_premium
                    ce_short_exit_price = ce_short_close
                    pe_short_exit_price = pe_short_close
                    ce_long_exit_price = ce_long_close
                    pe_long_exit_price = pe_long_close
                    exit_time = current_datetime
                    break

        if exit_reason == 'Held till expiry':
            exit_time = expiry_datetime

            options_expiry_data = options_df[(options_df['Expiry'] == trade['Expiry']) &
                                             (options_df['DateTime'] == expiry_datetime)]
            
            ce_short_exit_price = options_expiry_data[(options_expiry_data['Strike'] == trade['Ce_Short_Strike']) & (options_expiry_data['Type'] == 'CE')]['Close'].values[0]
            pe_short_exit_price = options_expiry_data[(options_expiry_data['Strike'] == trade['Pe_Short_Strike']) & (options_expiry_data['Type'] == 'PE')]['Close'].values[0]
            ce_long_exit_price = options_expiry_data[(options_expiry_data['Strike'] == trade['Ce_Long_Strike']) & (options_expiry_data['Type'] == 'CE')]['Close'].values[0]
            pe_long_exit_price = options_expiry_data[(options_expiry_data['Strike'] == trade['Pe_Long_Strike']) & (options_expiry_data['Type'] == 'PE')]['Close'].values[0]

            total_premium = (trade['Ce_Short_En_Price'] + trade['Pe_Short_En_Price'] - trade['Ce_Long_En_Price'] - trade['Pe_Long_En_Price']) - (ce_short_exit_price + pe_short_exit_price - ce_long_exit_price - pe_long_exit_price)
            total_pnl = total_premium_received - total_premium

        trade_log_df.at[index, 'Ce_Ex_Date'] = exit_time
        trade_log_df.at[index, 'Pe_Ex_Date'] = exit_time
        trade_log_df.at[index, 'Ce_Short_Ex_Price'] = ce_short_exit_price
        trade_log_df.at[index, 'Pe_Short_Ex_Price'] = pe_short_exit_price
        trade_log_df.at[index, 'Ce_Long_Ex_Price'] = ce_long_exit_price
        trade_log_df.at[index, 'Pe_Long_Ex_Price'] = pe_long_exit_price
        trade_log_df.at[index, 'Exit_Reason'] = exit_reason
        trade_log_df.at[index, 'Total_PnL'] = total_pnl

    return trade_log_df


In [6]:
def get_unique_dates(futures_df):
    """
    This function retrieves unique dates from the futures DataFrame.

    Parameters:
    futures_df (pd.DataFrame): Input DataFrame with 'Date' column

    Returns:
    pd.DataFrame: A DataFrame containing unique dates in the DataFrame
    """
    # Ensure 'Date' column is in datetime format
    futures_df['Date'] = pd.to_datetime(futures_df['Date'], format='%Y-%m-%d')

    # Retrieve unique dates
    unique_dates = futures_df['Date'].dt.date.unique()

    # Create a DataFrame with the unique dates
    unique_dates_df = pd.DataFrame(unique_dates, columns=['Date'])

    return unique_dates_df


In [7]:
import pandas as pd
import matplotlib.pyplot as plt

def calculate_performance_stats(trade_log_df):
    """
    Calculate performance statistics from the trade log.

    Parameters:
    trade_log_df (pd.DataFrame): DataFrame containing the trade log entries

    Returns:
    dict: Dictionary containing performance statistics
    """
    total_trades = len(trade_log_df)
    total_profit_loss = trade_log_df['Total_PnL'].sum()
    average_profit_loss = trade_log_df['Total_PnL'].mean()
    win_rate = (trade_log_df[trade_log_df['Total_PnL'] > 0].count()['Total_PnL'] / total_trades) * 100
    loss_rate = (trade_log_df[trade_log_df['Total_PnL'] < 0].count()['Total_PnL'] / total_trades) * 100
    max_drawdown = trade_log_df['Total_PnL'].min()
    max_profit = trade_log_df['Total_PnL'].max()
    
    return {
        'Total Trades': total_trades,
        'Total Profit/Loss': total_profit_loss,
        'Average Profit/Loss per Trade': average_profit_loss,
        'Win Rate (%)': win_rate,
        'Loss Rate (%)': loss_rate,
        'Max Drawdown': max_drawdown,
        'Max Profit': max_profit
    }


In [8]:
def plot_equity_curve(trade_log_df):
    """
    Plot the equity curve and drawdown from the trade log.

    Parameters:
    trade_log_df (pd.DataFrame): DataFrame containing the trade log entries
    """
    # Calculate cumulative profit/loss
    trade_log_df['Cumulative_PnL'] = trade_log_df['Total_PnL'].cumsum()
    
    # Calculate drawdown
    trade_log_df['Drawdown'] = trade_log_df['Cumulative_PnL'] - trade_log_df['Cumulative_PnL'].cummax()
    
    # Plot equity curve
    plt.figure(figsize=(12, 6))
    plt.plot(trade_log_df['Date'], trade_log_df['Cumulative_PnL'], label='Equity Curve')
    plt.plot(trade_log_df['Date'], trade_log_df['Drawdown'], label='Drawdown', color='red')
    plt.xlabel('Date')
    plt.ylabel('PnL')
    plt.title('Equity Curve and Drawdown')
    plt.legend()
    plt.grid(True)
    plt.savefig('equity_curve.png')
    plt.show()


In [9]:
import pandas as pd

def main():
    options_df = pd.read_csv(r"D:\AlgoT\DATA\BANKNIFTY_FNO_Data_2017\BANKNIFTY_2017_OPTIONS.csv")
    futures_df = pd.read_csv(r"D:\AlgoT\DATA\BANKNIFTY_FNO_Data_2017\BANKNIFTY_2017_FUTURES.csv")
    # Ensure 'Date' and 'Time' columns are in datetime format
    futures_df['Date'] = pd.to_datetime(futures_df['Date'], format='%Y-%m-%d')
    futures_df['Time'] = pd.to_datetime(futures_df['Time'], format='%H:%M:%S').dt.time
    options_df['Date'] = pd.to_datetime(options_df['Date'], format='%Y-%m-%d')
    options_df['Time'] = pd.to_datetime(options_df['Time'], format='%H:%M:%S').dt.time
    options_df['Expiry'] = pd.to_datetime(options_df['Expiry'], format='%Y-%m-%d')

    # Get unique dates from futures_df
    unique_dates_df = get_unique_dates(futures_df)

    # Print the DataFrame containing unique dates
    print("Unique Dates DataFrame:")
    print(unique_dates_df)

    # Define trading parameters
    entry_time = '10:29:59'
    total_capital = 100000  # Example capital amount

    # Initialize a DataFrame to collect all trade logs
    all_trade_logs = []
    error_log = []

    for _, row in unique_dates_df.iterrows():
        selected_date = row['Date']
        
    
        print(f"Processing date: {selected_date}")

        try:
            # Create straddle and log trades
            trade_log = create_straddle(options_df, futures_df, str(selected_date), entry_time, total_capital)

            # If trade log is not empty, append to all_trade_logs
            if trade_log:
                trade_log_df = pd.DataFrame(trade_log)

                # Monitor and exit trades based on conditions
                updated_trade_log_df = monitor_and_exit_trades(trade_log_df, options_df)

                all_trade_logs.append(updated_trade_log_df)
            else:
                error_log.append({'Date': selected_date, 'Error': 'No trades were logged'})
        except Exception as e:
            error_log.append({'Date': selected_date, 'Error': str(e)})
            print(f"Error processing date {selected_date}: {e}")

    if all_trade_logs:
        final_trade_log_df = pd.concat(all_trade_logs).reset_index(drop=True)
        final_trade_log_df.to_csv('final_trades_log.csv', index=False)
        print("Final trade log has been saved to final_trade_log.csv")
        
        # Calculate performance statistics
        performance_stats = calculate_performance_stats(final_trade_log_df)
        print("Performance Statistics:")
        for key, value in performance_stats.items():
            print(f"{key}: {value}")

        # Plot equity curve and drawdown
        plot_equity_curve(final_trade_log_df)
    else:
        print("No trades were logged.")

    if error_log:
        error_log_df = pd.DataFrame(error_log)
        error_log_df.to_csv('error_logs.csv', index=False)
        print("Error log has been saved to error_log.csv")
    else:
        print("No errors encountered.")

if __name__ == "__main__":
    main()

Unique Dates DataFrame:
           Date
0    2017-01-02
1    2017-01-03
2    2017-01-04
3    2017-01-05
4    2017-01-06
..          ...
243  2017-12-22
244  2017-12-26
245  2017-12-27
246  2017-12-28
247  2017-12-29

[248 rows x 1 columns]
Processing date: 2017-01-02
Processing date: 2017-01-03
Processing date: 2017-01-04


KeyboardInterrupt: 